In [1]:
import numpy as np
import copy
import math

# Parsing

In [2]:
with open("data/input20.txt") as f:
    data = f.read().rstrip("\n")

def parse(inlines):
    ls = {l.split(" -> ")[0][1:]: (l.split(" -> ")[0][0], l.split(" -> ")[1].split(", ")) for l in inlines.split("\n")}
    lsnew = {}
    for k,v in ls.items():
        if v[0]=="%":
            lsnew[k] = (v[0], "off", v[1])
        elif v[0]=="&":
            lsnew[k] = (v[0], {}, v[1])
        else:
            lsnew[k] = (v[0], None, v[1])
    for k,v in lsnew.items():
        for dest in v[2]:
            if dest in lsnew and lsnew[dest][0]=="&":
                lsnew[dest][1][k] = "low"
    return lsnew

ls = parse(data)

# Part 1

In [3]:
lows = 0
highs = 0
ls2 = copy.deepcopy(ls)

for push in range(1000):
    signals = [("roadcaster", "low")]
    while signals:
        newsignals = []
        for id, signal in signals:
            if signal=="low":
                lows += 1
            else:
                highs += 1
            if not id in ls2:
                continue
            mtype = ls2[id][0]
            if signal=="high":
                pass
            else:
                if ls2[id][1]=="off":
                    ls2[id] = (ls2[id][0], "on", ls2[id][2])
                elif ls2[id][1]=="on":
                    ls2[id] = (ls2[id][0], "off", ls2[id][2])
            for newid in ls2[id][2]:
                signalsent = False
                if mtype=="%":
                    if signal=="high":
                        pass
                    else:
                        if ls2[id][1]=="off":
                            newsignals.append((newid, "low"))
                            signalsent = True
                        else:
                            newsignals.append((newid, "high"))
                            signalsent = True
                elif mtype=="&":
                    if all([sig=="high" for _,sig in ls2[id][1].items()]):
                        newsignals.append((newid, "low"))
                        signalsent = True
                    else:
                        newsignals.append((newid, "high"))
                        signalsent = True
                elif mtype=="b":
                    newsignals.append((newid, "low"))
                    signalsent = True
                if newid in ls2 and signalsent:
                    newtype = ls2[newid][0]
                    if newtype=="&":
                        ls2[newid][1][id] = newsignals[-1][1]
        signals = newsignals
print(lows*highs)

807069600


# Part 2

In [5]:
lows = 0
highs = 0
ls2 = copy.deepcopy(ls)
found = False
iter = 0
rxs = {id: [] for id, _ in ls2["hf"][1].items()}
while not all([len(l)>=1 for _, l in rxs.items()]): # First time reaching hf as low is the same as the periodicity.
    iter += 1
    signals = [("roadcaster", "low")]
    while signals:
        newsignals = []
        for id, signal in signals:
            if id=="rx" and signal=="low":
                print("Found it!")
                found = True
                newsignals=[]
                break
            if signal=="low":
                lows += 1
            else:
                highs += 1
            if not id in ls2:
                continue
            mtype = ls2[id][0]
            if signal=="high":
                pass
            else:
                if ls2[id][1]=="off":
                    ls2[id] = (ls2[id][0], "on", ls2[id][2])
                elif ls2[id][1]=="on":
                    ls2[id] = (ls2[id][0], "off", ls2[id][2])
            for newid in ls2[id][2]:
                signalsent = False
                if mtype=="%":
                    if signal=="high":
                        pass
                    else:
                        if ls2[id][1]=="off":
                            newsignals.append((newid, "low"))
                            signalsent = True
                        else:
                            newsignals.append((newid, "high"))
                            signalsent = True
                elif mtype=="&":
                    if all([sig=="high" for _,sig in ls2[id][1].items()]):
                        newsignals.append((newid, "low"))
                        signalsent = True
                    else:
                        newsignals.append((newid, "high"))
                        signalsent = True
                elif mtype=="b":
                    newsignals.append((newid, "low"))
                    signalsent = True
                if newid in ls2 and signalsent:
                    newtype = ls2[newid][0]
                    if newtype=="&":
                        ls2[newid][1][id] = newsignals[-1][1]
        signals = newsignals
        for id, val in ls2["hf"][1].items():
            if val=="high":
                rxs[id].append(iter)
assert np.gcd.reduce((np.array([v[0] for k,v in rxs.items()])).astype(int))==1 # gcd is 1
accum = 1
for k,v in rxs.items():
    accum*=v[0]
accum

221453937522197